# Question Answering Machine

### Final Project for AMLPP

### Carlos Grandet & Hector Salvador

The following is a demo for a question answering machine (QAM) that we build for a machine learning class. The QAM uses several NLP tools such as Stanford POSTagger, Word2Vec and 
BM25. 

In [67]:
# First step, load the relevant libraries and classes

from question_processing import Question
from synonyms import Synonyms
from Word2VecModel import W2V_Model 
from spanish_tagger import Spanish_Postagger
from query import Query
from index import go
from retrieval import ScoreParagraphs
from ngrams import retrieve_model
import json
import math

# Then name the different files you will use to make the code work

tagfile = 'stanford-postagger-full-2016-10-31/models/spanish.tagger'
jarfile = 'stanford-postagger-full-2016-10-31/stanford-postagger.jar'

# Data for synonyms API
APIfile = 'http://store.apicultur.com/api/sinonimosporpalabra/1.0.0/' 
token = 'f7JE_2svUVwP5ARGfw8aQhnLXlga'

# Data for w2v model
w2vfile = 'SBW-vectors-300-min5.txt'

# Data for question_type
question_json = 'Data/question_type.json'
stopwords = 'Data/stopwords.json'


### Create a Query class, which allows you to make questions, get queries and find related words. 

Creating this query might take a while, since you need to load a Word2Vec model that is over 2.5 GB!!! We advise running the following line of code only once, as it might take around 10 minutes for it to run.

## RUN ONCE

In [7]:
query = Query(APIfile, token, tagfile, jarfile, 
    w2vfile)

Once we have created the Query class, we can start asking various questions

In [69]:
question = "¿Cuál es el castigo por homicidio?" #Translation: What is the punishment for murder?

#set_question loads a question and a json file with info about the words that represent 
#a question type (i.e. time: years, period, days, etc )
query.set_question(question, question_json)

#get_query gives you a specific query list
query.get_query(stopwords)


http://store.apicultur.com/api/sinonimosporpalabra/1.0.0/homicidio
Bearer f7JE_2svUVwP5ARGfw8aQhnLXlga


You can access the query selected for that question and the question type with 
the following attributes

In [70]:
print(query.query)
print(query.qtype)

{'culposo', 'crimen', 'doloso', 'homicidio', 'delito', 'asesinato', 'homicido'}
['años']


You can also add words to the query, if you want

In [71]:
query.add_words(["asesinato", "imprudencial"])

You can also remove words

In [73]:
query.remove_words(["asesinato", "doloso"])

In [74]:
query.query

{'crimen', 'culposo', 'delito', 'homicidio', 'homicido', 'imprudencial'}

Finally, you can play with word to vec to find other words that could match your query or find words that do not belong there

In [13]:
query.W2V.find_concepts(positive = ["rey", "mujer"], negative = ["hombre"], top_n = 1)

['reina']

In [14]:
query.W2V.intruder(["rana", "pato", "simio", "pelota"])

'pelota'

### Once you have a query, the next step is to use information retrieval to find the relevant documents

We will assume that the user has: the .txt documents in a folder called "./leyes", the "docnames.csv" file in a folder called "./doc". Run first the following script to create a set of indices to make queries efficient. Note that this can take up to 5 minutes. 

## RUN IT ONLY ONCE

In [19]:
go()

Using stemmed words.
Processing 2_241213..
  ..finished with 2_241213.
Processing 3_250117..
  ..finished with 3_250117.
Processing 4_160516..
  ..finished with 4_160516.
Processing 6..
  ..finished with 6.
Processing 8_270117..
  ..finished with 8_270117.
Processing CMPP..
  ..finished with CMPP.
Processing CNPP_170616..
  ..finished with CNPP_170616.
Processing 9_180716..
  ..finished with 9_180716.
Processing 10_270614..
  ..finished with 10_270614.
Processing 11..
  ..finished with 11.
Processing 79..
  ..finished with 79.
Processing 12_270117..
  ..finished with 12_270117.
Processing 13_191216..
  ..finished with 13_191216.
Processing 14_101114..
  ..finished with 14_101114.
Processing 15_080616..
  ..finished with 15_080616.
Processing 16_240316..
  ..finished with 16_240316.
Processing 17..
  ..finished with 17.
Processing 19..
  ..finished with 19.
Processing LAmp_170616..
  ..finished with LAmp_170616.
Processing 21..
  ..finished with 21.
Processing 22_270117..
  ..finished w

The following code prioritizes the articles that are relevant to our search, based on the words provided by the query.

In [75]:
top_results = 20
words = list(query.query)
test = ScoreParagraphs(question, words, stem=True)
results = test.texts(top_results, method='bm25')

Working with stemmed words: ['culpos', 'crim', 'imprudencial', 'homicidi', 'delit', 'homic']
                                                 text     score  \
0    309 Se deroga\n   \nCAPITULO III\nReglas comu...  1.838249   
1    313 Si el occiso o suicida fuere menor de eda...  1.614150   
2    59 Bis Se deroga\n   \nCAPITULO II\nAplicació...  1.512563   
3    302 Comete el delito de homicidio el que priv...  1.497126   
4    320 Al responsable de un homicidio calificado...  1.252807   
5    307 Al responsable de cualquier homicidio sim...  0.786295   
6    301 De las lesiones que a una persona cause a...  0.673755   
7    137 Cuando durante una rebelión se cometan lo...  0.616268   
8    315 Bis Se impondrá la pena del artículo 320 ...  0.538570   
9    308 Si el homicidio se comete en riña se apli...  0.516264   
10   315 Se entiende que las lesiones y el homicid...  0.463249   
11   61 En los casos a que se refiere la primera p...  0.453452   
12   322 Además de las sanciones que

We could manually inspect some of these results, but that could potentially take a lot of time...

In [76]:
for i in results.text:
    print(i)
    print("--------------")

 309 Se deroga
   
CAPITULO III
Reglas comunes para lesiones y homicidio
   
   
--------------
 313 Si el occiso o suicida fuere menor de edad o padeciere alguna de las formas de enajenación mental se aplicarán al homicida o instigador las sanciones señaladas al homicidio calificado o a las lesiones calificadas
   
   
--------------
 59 Bis Se deroga
   
CAPITULO II
Aplicación de sanciones a los delitos culposos
   
   
--------------
 302 Comete el delito de homicidio el que priva de la vida a otro
   
   
--------------
 320 Al responsable de un homicidio calificado se le impondrán de treinta a sesenta años de prisión
   
   
--------------
 307 Al responsable de cualquier homicidio simple intencional que no tenga prevista una sanción especial en este Código se le impondrán de doce a veinticuatro años de prisión
   
   
--------------
 301 De las lesiones que a una persona cause algún animal bravío será responsable el que con esa intención lo azuce o lo suelte o haga esto último po

Let's instead build an n-gram model that attempts to predict which of these look like articles that have a answer with sanctions, penalties, of fines. Then let's get those probabilities and show only the results that have a high probability of being an answer.

In [77]:
clf = retrieve_model()
clfs = clf.predict(results)
probs = [math.exp(i)-1 for i in clf.predict(results)]

In [78]:
print("Maybe these are your results...\n")
for i in range(len(results.text)):
    if probs[i] > 0:
        print("--------------")
        print(results.text[i])

Maybe these are your results...

--------------
 60 En los casos de delitos culposos se impondrá hasta la cuarta parte de las penas y medidas de seguridad asignadas por la ley al tipo básico del delito doloso con excepción de aquéllos para los que la ley señale una pena específica Además se impondrá en su caso suspensión hasta de tres años de derechos para ejercer profesión oficio autorización licencia o permiso
   
   Las sanciones por delitos culposos sólo se impondrán en relación con los delitos previstos en los siguientes artículos 150 167 fracción VI 169 199 Bis 289 parte segunda 290 291 292 293 302 307 323 397 399 414 primer párrafo y tercero en su hipótesis de resultado 415 fracciones I y II y último párrafo en su hipótesis de resultado 416 420 fracciones I II III y V y 420 Bis fracciones I II y IV de este Código
   
   Cuando a consecuencia de actos u omisiones culposos calificados como graves que sean imputables al personal que preste sus servicios en una empresa ferroviaria a

## Let's try a query of your own!

In [ ]:
# question = "¿Cuál es el castigo por extraer hidrocarburos ilegalmente?" 
# Translation: What is penalty for illegaly extracting hydrocarbons?

In [118]:
question = "¿Cuál es la multa por traición a la patria?" 
# Translation: What is penalty for not paying taxes?

In [119]:
query.set_question(question, question_json)
query.get_query(stopwords)
query.add_words(["multa", "sanción"])
words = list(query.query)
words

http://store.apicultur.com/api/sinonimosporpalabra/1.0.0/traición
Bearer f7JE_2svUVwP5ARGfw8aQhnLXlga
http://store.apicultur.com/api/sinonimosporpalabra/1.0.0/patria
Bearer f7JE_2svUVwP5ARGfw8aQhnLXlga


['nación',
 'exclamen',
 'patriotismo',
 'Patria',
 'alevosía',
 'ciudadanía',
 'patria',
 'sanción',
 'traición',
 'deslealtad',
 'nacionalidad',
 'legarnos',
 'multa']

In [120]:
test = ScoreParagraphs(question, words, stem=True)
results = test.texts(top_results, method='bm25')

Working with stemmed words: ['nacion', 'exclam', 'patriot', 'patri', 'alev', 'ciudadan', 'patri', 'sancion', 'traicion', 'deslealt', 'nacional', 'leg', 'mult']
                                                 text     score  \
0    122 Se deroga\n   \nLIBRO SEGUNDO\n\nTITULO P...  4.676978   
1    319 Se dice que obra a traición el que no sol...  1.535522   
2    318 La alevosía consiste en sorprender intenc...  1.317167   
3    295 Al que ejerciendo la patria potestad o la...  1.089244   
4    409 Se impondrán de veinte a cien días multa ...  1.033274   
5    206 El lenocinio se sancionará con prisión de...  0.983053   
6    315 Se entiende que las lesiones y el homicid...  0.838950   
7    335 Al que abandone a un niño incapaz de cuid...  0.788663   
8    376 En todo caso de robo si el juez lo creyer...  0.758250   
9    97 Cuando la conducta observada por el senten...  0.614068   
10   366 Ter Comete el delito de tráfico de menore...  0.608689   
11   128 Se aplicará la pena de pris

In [121]:
clf = retrieve_model()
clfs = clf.predict(results)
probs = [math.exp(i)-1 for i in clf.predict(results)]

In [122]:
print("Maybe these are your results...\n")
for i in range(len(results.text)):
    if probs[i] > 0:
        print("--------------")
        print(results.text[i])

Maybe these are your results...

--------------
 97 Cuando la conducta observada por el sentenciado refleje un alto grado de reinserción social y su liberación no represente un riesgo para la tranquilidad y seguridad públicas conforme al dictamen del órgano ejecutor de la sanción y no se trate de sentenciado por traición a la Patria espionaje terrorismo sabotaje genocidio delitos contra la salud violación delito intencional contra la vida y secuestro desaparición forzada tortura y trata de personas ni de reincidente por delito intencional se le podrá conceder indulto por el Ejecutivo Federal en uso de facultades discrecionales expresando sus razones y fundamentos en los casos siguientes
   
   I Por los delitos de carácter político a que alude el artículo 144 de este Código
   
   II Por otros delitos cuando la conducta de los responsables haya sido determinada por motivaciones de carácter político o social y
   
   III Por delitos de orden federal o común en el Distrito Federal cuando